In [6]:
import requests
import re
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pandas as pd
import os

In [3]:
def get_HTML_text(page, url, failList, timeGap, path):
#     kv = {'User-Agent': 'Mozilla/5.0'}
    try:
        time.sleep(timeGap)
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        print(page + " fail." + '\r\n')
        saveFailedMovie(failList, page, path)

In [4]:
def saveFailedPage(faillist, page, path):
    faillist.append(page)
    filename = os.path.join(path, str('failpageList.txt'))
    f = open(filename, 'wt+')
    f.write(str(page) + '\n')
    f.close()

In [16]:
def get_budget(html):
    budgets = []
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find('table')
    html = get_HTML_text(page, url, failList, 1, '')
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find('table')
    table_rows = table.find_all('tr')
    for index, tr in enumerate(table_rows):
        if index == 0:
            continue
        else:
            td = tr.find_all('td')
            row = [i.text for i in td]
            date = str(datetime.strptime(row[1], '%b %d, %Y').year)
            name = row[2]
            prod_budget = row[3].replace('\xa0$', '').replace(',', '')
            domestic_gross = row[4].replace('\xa0$', '').replace(',', '')
            worldwide_gross = row[5].replace('\xa0$', '').replace(',', '')
            budget = Budget(name, date, prod_budget, domestic_gross, worldwide_gross)
            budgets.append(budget)
    return budgets

In [11]:
failList = []
page = '1'
url = 'https://www.the-numbers.com/movie/budgets/all/1'
html = get_HTML_text(page, url, failList, 1, '')
soup = BeautifulSoup(html, "html.parser")
table = soup.find('table')
table_rows = table.find_all('tr')
for index, tr in enumerate(table_rows):
    if index == 0:
        continue
    else:
        td = tr.find_all('td')
        row = [i.text for i in td]
        date = str(datetime.strptime(row[1], '%b %d, %Y').year)
        name = row[2]
        prod_budget = row[3].replace('\xa0$', '').replace(',', '')
        domestic_gross = row[4].replace('\xa0$', '').replace(',', '')
        worldwide_gross = row[5].replace('\xa0$', '').replace(',', '')
        budget = [name, date, prod_budget, domestic_gross, worldwide_gross]
        print(budget)

['Avengers: Endgame', '2019', '400000000', '858373000', '2797800564']
['Pirates of the Caribbean: On Stranger Tides', '2011', '379000000', '241063875', '1045663875']
['Avengers: Age of Ultron', '2015', '365000000', '459005868', '1403013963']
['Star Wars Ep. VII: The Force Awakens', '2015', '306000000', '936662225', '2068223624']
['Avengers: Infinity War', '2018', '300000000', '678815482', '2048359754']
['Pirates of the Caribbean: At World’s End', '2007', '300000000', '309420425', '963420425']
['Justice League', '2017', '300000000', '229024295', '655945209']
['Spectre', '2015', '300000000', '200074175', '879620923']
['Star Wars: The Rise of Skywalker', '2019', '275000000', '515202542', '1074141030']
['Solo: A Star Wars Story', '2018', '275000000', '213767512', '393151347']
['John Carter', '2012', '263700000', '73058679', '282778100']
['Batman v Superman: Dawn of Justice', '2016', '263000000', '330360194', '873634919']
['The Lion King', '2019', '260000000', '543638043', '1656943394']
['T

In [1]:
class Budget:
    def __init__(self, name, date, prod_budget, domestic_gross, worldwide_gross):
        self.name = name
        self.date = date
        self.prod_budget = prod_budget
        self.domestic_gross = domestic_gross
        self.worldwide_gross = worldwide_gross

In [18]:
def scrape_plot(output_file, path):
    failList = []  # collect page that failed
    with open(output_file, 'a+') as output_f:
        for page in range(6001, 6101, 100):
            print('starting scraping page {}'.format(page))
            url = 'https://www.the-numbers.com/movie/budgets/all/{}'.format(page)
            html = get_HTML_text(page, url, failList, 1, path)
            budgets = get_budget(html)
            for budget in budgets:
                output_f.write('{}\t{}\t{}\t{}\t{}\n'.format(budget.name, budget.date, budget.prod_budget, budget.domestic_gross, budget.worldwide_gross))

In [19]:
scrape_plot('../../Sample_Data/Processed/budget.tsv', '')

starting scraping page 6001
